In [29]:
%env PYTORCH_ENABLE_MPS_FALLBACK=1

from tqdm.notebook import tqdm
import wandb

from WindModel import *

wandb.login()

env: PYTORCH_ENABLE_MPS_FALLBACK=1


True

In [30]:
DEVICE = "cpu"

In [31]:
WindDataset.init(0.1)

train = WindDataset("train")
validation = WindDataset("validation")
test = WindDataset("test")

del WindDataset.data

In [32]:
INPUT_SIZE = 15

EPOCHS = 30

LOSS_FUNC = torch.nn.MSELoss

In [39]:
class SinusoidalLayer(torch.nn.Module):
  def __init__(self, size: int):
    super().__init__()
    self.amplitudes = torch.nn.Parameter(torch.rand(size))
    self.periods = torch.nn.Parameter(torch.rand(size))
    self.phases = torch.nn.Parameter(torch.rand(size))

  def forward(self, x: torch.Tensor):
    x *= self.periods
    x += self.phases
    x = torch.sin(x)
    x *= self.amplitudes

    return torch.sum(x, dim=1)


class WindModel(torch.nn.Module):
    def __init__(self, input_size, sin_size: int):
        super().__init__()

        self.linear = torch.nn.Linear(input_size, sin_size)
        self.sin_layer = SinusoidalLayer(sin_size)

    def forward(self, x: torch.Tensor):
        z = self.linear(x)
        z = self.sin_layer(z)

        return z + x[:, 3]


In [40]:
def evaluate_one_epoch(model, epoch, dl):
    mse = 0
    mae = 0

    with torch.no_grad():
        for inputs, targets in dl:
            prediction = model(inputs).squeeze()

            mse += torch.nn.functional.mse_loss(prediction, targets)
            mae += torch.nn.functional.l1_loss(prediction, targets)

    n = len(dl)
    wandb.log({"val_rmse": ((mse / n) ** 0.5) * stds[VARIABLE]})
    wandb.log({"val_mae": (mae / n) * stds[VARIABLE]})


def train_one_batch(model, optimizer, criterion, batch, batch_idx):
    optimizer.zero_grad()

    inputs, targets = batch

    prediction = model(inputs).squeeze()
    loss = criterion(prediction, targets)

    loss.backward()
    optimizer.step()

    if batch_idx % 100 == 0:
        with torch.no_grad():
            rmse = (torch.nn.functional.mse_loss(prediction, targets) ** 0.5) * stds[VARIABLE]
            wandb.log({"train_loss": loss})
            wandb.log({"train_rmse": rmse})


def train_one_epoch(model, optimizer, criterion, epoch, dl):
    data = iter(dl)
    for i in range(len(dl)):
        train_one_batch(model, optimizer, criterion, next(data), i)


def main():
    wandb.init(project=f"MERRA2-{VARIABLE}", dir="wandb-local")

    learning_rate = wandb.config.learning_rate
    batch_size = wandb.config.batch_size
    layers = wandb.config.layers

    train_dl = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=10)
    validation_dl = DataLoader(validation, batch_size=batch_size, shuffle=False, num_workers=10)

    model = WindModel(INPUT_SIZE, layers)
    model = model.to(DEVICE)
    print(model)

    criterion = LOSS_FUNC()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    wandb.watch(model, log_freq=100)
    model.train()

    for ep in tqdm(range(EPOCHS)):
        print(ep, end=" ")
        train_one_epoch(model, optimizer, criterion, ep, train_dl)
        evaluate_one_epoch(model, ep, validation_dl)

    wandb.finish()

In [41]:
sweep_configuration = {
    "method": "bayes",
    "name": f"sweep-SNN-{DATASET}",
    "metric": {
        "goal": "minimize",
        "name": "val_rmse"
    },
    "parameters": {
        "batch_size": {"value": 8192},
        "learning_rate": {"max": 0.1, "min": 0.0001},
        "layers": {"values": [1024]},
        "estimate_quantile": {"value": ESTIMATE_QUANTILE},
        "dataset": {"value": DATASET},
        "epochs": {"value": EPOCHS},
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 3,
    }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project=f"MERRA2-{VARIABLE}")
wandb.agent(sweep_id, function=main)


Create sweep with ID: wss2uso4
Sweep URL: https://wandb.ai/bhavye-mathur/MERRA2-U/sweeps/wss2uso4


wandb: Agent Starting Run: 2ykotlk6 with config:
wandb: 	batch_size: 8192
wandb: 	dataset: NGCT
wandb: 	epochs: 30
wandb: 	estimate_quantile: 0.9935
wandb: 	layers: 1024
wandb: 	learning_rate: 0.01145626386197857
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Path wandb-local/wandb/ wasn't writable, using system temp directory


WindModel(
  (linear): Linear(in_features=15, out_features=1024, bias=True)
  (sin_layer): SinusoidalLayer()
)


  0%|          | 0/30 [00:00<?, ?it/s]

0 1 2 3 4 5 6 7 8 

train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▅▃▁▁▃▇▄
val_rmse,█▅▃▂▁▃▆▃
train_loss,0.01613
train_rmse,1.24852
val_mae,0.95883
val_rmse,1.22906
